In [104]:
import requests
import pandas as pd
import xmltodict
from os import name
import xml.etree.ElementTree as et
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

url = "https://safemap.go.kr/openApiService/data/getCrmnlStatsMurderData.do"
params = {
"serviceKey": "",
"numOfRows": "3000",
"pageNo": "1",
"dataType": "xml"
}

response = requests.get(url, params=params)
content = response.text

xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('item')

row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

# xml 안의 데이터 수집
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

#xml값 DataFrame으로 만들기
df = pd.DataFrame(row_list, columns=name_list)
df

,OBJT_ID,POLC_NM,POLC_SE,PLCSTN_NM,MURDER,CTPRVN_NM,SGG_KOR_NM,CTPRVN_CD,SGG_CD,X,Y
0,1,서울서부,01_경찰서,형사지원팀,1,서울특별시,은평구,11,11380,14128814.2085,4523368.85601
1,2,서울영등포,01_경찰서,다목적기동순찰대,1,서울특별시,영등포구,11,11560,14126536.9234,4512701.85245
2,3,서울강남,01_경찰서,다목적기동순찰대,1,서울특별시,강남구,11,11680,14145045.9595,4510321.54401
3,4,부산동부,01_경찰서,형사1팀,1,부산광역시,동구,26,26170,14365688.5262,4181553.41277
4,5,대전둔산,01_경찰서,다목적기동순찰대,1,대전광역시,서구,30,30170,14179932.5417,4350045.11408
...,...,...,...,...,...,...,...,...,...,...,...
2037,2038,서귀포,03_파출소,대정,3,제주특별자치도,서귀포시,50,50130,14054773.656,3924933.67007
2038,2039,서귀포,03_파출소,중문,2,제주특별자치도,서귀포시,50,50130,14073291.4084,3928849.47696
2039,2040,서귀포,03_파출소,대신,3,제주특별자치도,서귀포시,50,50130,14082991.9892,3929113.79376
2040,2041,서귀포,03_파출소,효돈,1,제주특별자치도,서귀포시,50,50130,14094955.7175,3930378.55343


In [105]:
df.set_index("OBJT_ID",inplace=True)

In [106]:
import pyproj
lat=[]
lot=[]
transformer = pyproj.Transformer.from_crs(pyproj.CRS('epsg:3857'),pyproj.CRS('epsg:4326'))
for i in df.index:
    df0 = df[df.index == i]
    x,y=transformer.transform(df0.X,df0.Y)
    la=str(x)
    lo=str(y)
    la=la.replace('[', '').replace(']', '')
    lo=lo.replace('[', '').replace(']', '')
    lat.append(la)
    lot.append(lo)
df['latitude']=lat
df['longitude']=lot

In [107]:
df

,POLC_NM,POLC_SE,PLCSTN_NM,MURDER,CTPRVN_NM,SGG_KOR_NM,CTPRVN_CD,SGG_CD,X,Y,latitude,longitude
OBJT_ID,,,,,,,,,,,,
1,서울서부,01_경찰서,형사지원팀,1,서울특별시,은평구,11,11380,14128814.2085,4523368.85601,37.6021105,126.9212975
2,서울영등포,01_경찰서,다목적기동순찰대,1,서울특별시,영등포구,11,11560,14126536.9234,4512701.85245,37.5261541,126.9008403
3,서울강남,01_경찰서,다목적기동순찰대,1,서울특별시,강남구,11,11680,14145045.9595,4510321.54401,37.5091941,127.0671098
4,부산동부,01_경찰서,형사1팀,1,부산광역시,동구,26,26170,14365688.5262,4181553.41277,35.1299392,129.0491757
5,대전둔산,01_경찰서,다목적기동순찰대,1,대전광역시,서구,30,30170,14179932.5417,4350045.11408,36.3583691,127.3805013
...,...,...,...,...,...,...,...,...,...,...,...,...
2038,서귀포,03_파출소,대정,3,제주특별자치도,서귀포시,50,50130,14054773.656,3924933.67007,33.2229447,126.2561799
2039,서귀포,03_파출소,중문,2,제주특별자치도,서귀포시,50,50130,14073291.4084,3928849.47696,33.2523663,126.4225277
2040,서귀포,03_파출소,대신,3,제주특별자치도,서귀포시,50,50130,14082991.9892,3929113.79376,33.2543519,126.5096695


In [5]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import json
import pandas as pd

# Firebase 서비스 계정 키 파일 경로
cred = credentials.Certificate("")

# Firebase 앱 초기화
firebase_admin.initialize_app(cred)

db = firestore.client()

In [6]:
for col in df.columns:
    doc_ref=db.collection('crime').document(col)
    doc_ref.set(df[col].to_dict())

In [7]:
docs=db.collection('crime').stream()
data={}
for doc in docs:
    data[doc.id]=dict(sorted(doc.to_dict().items())).values()
sorted_df=pd.DataFrame(data, index=dict(sorted(doc.to_dict().items())).keys())
sorted_df

,CTPRVN_CD,CTPRVN_NM,MURDER,PLCSTN_NM,POLC_NM,POLC_SE,SGG_CD,SGG_KOR_NM,X,Y,latitude,longitude
1,11,서울특별시,1,형사지원팀,서울서부,01_경찰서,11380,은평구,14128814.2085,4523368.85601,37.6021105,126.9212975
10,48,경상남도,1,형사4팀,마산중부,01_경찰서,48125,창원시 마산합포구,14311694.2206,4190028.90847,35.1921837,128.5641366
100,11,서울특별시,3,도곡,서울수서,02_지구대,11680,강남구,14142441.9183,4509218.02792,37.5013301,127.0437173
1000,41,경기도,3,금곡,남양주남부,03_파출소,41360,남양주시,14160897.3541,4527902.6258,37.6343706,127.2095053
1001,41,경기도,2,다산1,남양주남부,03_파출소,41360,남양주시,14155619.1011,4524289.59293,37.60866315,127.16208995
...,...,...,...,...,...,...,...,...,...,...,...,...
995,41,경기도,3,가좌,일산서부,03_파출소,41287,고양시 일산서구,14106767.9288,4535912.91489,37.6913337,126.7232524
996,41,경기도,2,수택,구리,03_파출소,41310,구리시,14153958.2202,4522579.05096,37.5964892,127.14717
997,41,경기도,5,화도,남양주남부,03_파출소,41360,남양주시,14171607.803,4530142.41723,37.6503027,127.3057189
998,41,경기도,2,천마,남양주남부,03_파출소,41360,남양주시,14169480.9328,4531466.81391,37.6597218,127.2866129


In [8]:
sorted_df.index=sorted_df.index.astype(int)
sorted_df=sorted_df.sort_index()
sorted_df

,CTPRVN_CD,CTPRVN_NM,MURDER,PLCSTN_NM,POLC_NM,POLC_SE,SGG_CD,SGG_KOR_NM,X,Y,latitude,longitude
1,11,서울특별시,1,형사지원팀,서울서부,01_경찰서,11380,은평구,14128814.2085,4523368.85601,37.6021105,126.9212975
2,11,서울특별시,1,다목적기동순찰대,서울영등포,01_경찰서,11560,영등포구,14126536.9234,4512701.85245,37.5261541,126.9008403
3,11,서울특별시,1,다목적기동순찰대,서울강남,01_경찰서,11680,강남구,14145045.9595,4510321.54401,37.5091941,127.0671098
4,26,부산광역시,1,형사1팀,부산동부,01_경찰서,26170,동구,14365688.5262,4181553.41277,35.1299392,129.0491757
5,30,대전광역시,1,다목적기동순찰대,대전둔산,01_경찰서,30170,서구,14179932.5417,4350045.11408,36.3583691,127.3805013
...,...,...,...,...,...,...,...,...,...,...,...,...
2038,50,제주특별자치도,3,대정,서귀포,03_파출소,50130,서귀포시,14054773.656,3924933.67007,33.2229447,126.2561799
2039,50,제주특별자치도,2,중문,서귀포,03_파출소,50130,서귀포시,14073291.4084,3928849.47696,33.2523663,126.4225277
2040,50,제주특별자치도,3,대신,서귀포,03_파출소,50130,서귀포시,14082991.9892,3929113.79376,33.2543519,126.5096695
2041,50,제주특별자치도,1,효돈,서귀포,03_파출소,50130,서귀포시,14094955.7175,3930378.55343,33.2638524,126.6171415


In [75]:
df=sorted_df

In [108]:
df['MURDER'] = df['MURDER'].astype(float)
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

In [109]:
sido_df = df.groupby(['CTPRVN_CD', 'CTPRVN_NM']).agg({'MURDER': 'mean', 'latitude': 'mean', 'longitude': 'mean'}).reset_index()
sido_df['MURDER'] = sido_df['MURDER'].round(2)
sido_df

,CTPRVN_CD,CTPRVN_NM,MURDER,latitude,longitude
0,11,서울특별시,2.37,37.553376,126.994424
1,26,부산광역시,2.45,35.162078,129.052687
2,27,대구광역시,2.18,35.850727,128.568414
3,28,인천광역시,2.35,37.524796,126.557951
4,29,광주광역시,1.72,35.160248,126.862057
5,30,대전광역시,2.56,36.345617,127.394556
6,31,울산광역시,2.35,35.541147,129.311694
7,36,세종특별자치시,2.22,36.551364,127.281577
8,41,경기도,2.21,37.452016,127.063392
9,42,강원도,1.73,37.684190,128.351515


In [110]:
import numpy as np
OBJT_ID=np.arange(len(sido_df))
sido_df['OBJT_ID']=OBJT_ID
sido_df=sido_df.astype(str)
sido_df.set_index("OBJT_ID",inplace=True)

In [111]:
for col in sido_df.columns:
    doc_ref=db.collection('sidocrime').document(col)
    doc_ref.set(sido_df[col].to_dict())

In [112]:
docs=db.collection('sidocrime').stream()
data={}
for doc in docs:
    data[doc.id]=dict(sorted(doc.to_dict().items())).values()
sorted_df=pd.DataFrame(data, index=dict(sorted(doc.to_dict().items())).keys())
sorted_df

,CTPRVN_CD,CTPRVN_NM,MURDER,latitude,longitude
0,11,서울특별시,2.37,37.55337567195918,126.99442370416327
1,26,부산광역시,2.45,35.16207831189474,129.0526867768421
10,43,충청북도,1.76,36.72374373316456,127.73702579253164
11,44,충청남도,1.82,36.56093805573913,126.8681486098261
12,45,전라북도,1.58,35.76440147993827,127.03108608098765
13,46,전라남도,1.5,34.84213210758454,126.89502908995169
14,47,경상북도,1.42,36.26425489672489,128.7898710695633
15,48,경상남도,1.78,35.24119357982955,128.40427602784092
16,50,제주특별자치도,2.65,33.434829400384615,126.5348368646154
2,27,대구광역시,2.18,35.85072691129032,128.56841387741935


In [113]:
sig_df = df.groupby(['SGG_CD', 'SGG_KOR_NM']).agg({'MURDER': 'mean', 'latitude': 'mean', 'longitude': 'mean'}).reset_index()
sig_df['MURDER'] = sig_df['MURDER'].round(2)
sig_df

,SGG_CD,SGG_KOR_NM,MURDER,latitude,longitude
0,11110,종로구,1.70,37.579276,126.986495
1,11140,중구,1.60,37.562051,126.989678
2,11170,용산구,2.57,37.534339,126.984386
3,11200,성동구,2.33,37.553683,127.036506
4,11215,광진구,1.90,37.550042,127.080935
...,...,...,...,...,...
245,48870,함양군,1.29,35.539052,127.729294
246,48880,거창군,1.00,35.711320,127.912193
247,48890,합천군,1.14,35.593515,128.130307
248,50110,제주시,2.65,33.514401,126.511936


In [114]:
import numpy as np
OBJT_ID=np.arange(len(sig_df))
sig_df['OBJT_ID']=OBJT_ID
sig_df=sig_df.astype(str)
sig_df.set_index("OBJT_ID",inplace=True)

In [115]:
for col in sig_df.columns:
    doc_ref=db.collection('sigcrime').document(col)
    doc_ref.set(sig_df[col].to_dict())

In [116]:
docs=db.collection('sigcrime').stream()
data={}
for doc in docs:
    data[doc.id]=dict(sorted(doc.to_dict().items())).values()
sorted_df=pd.DataFrame(data, index=dict(sorted(doc.to_dict().items())).keys())
sorted_df

,MURDER,SGG_CD,SGG_KOR_NM,latitude,longitude
0,1.7,11110,종로구,37.579276335,126.986494625
1,1.6,11140,중구,37.562050920000004,126.98967816000001
10,2.88,11350,노원구,37.6526804625,127.0678840625
100,2.0,41430,의왕시,37.36397935,126.9729678
101,1.33,41450,하남시,37.53280903333333,127.191287
...,...,...,...,...,...
95,2.2,41310,구리시,37.60397768,127.13716661999999
96,2.06,41360,남양주시,37.65197976764706,127.21255950117647
97,3.0,41370,오산시,37.158745374999995,127.063547725
98,2.58,41390,시흥시,37.38548625,126.78065986666667
